<a href="https://colab.research.google.com/github/ulubeykhuja/Amaliyot/blob/main/salom_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sequence = 'salom'
chars = sorted(list(set(sequence)))
print(chars)
# char_to_idx = {char: idx for idx, char in enumerate(chars)}
# idx_to_char = {idx: char for idx, char in enumerate(chars)}

['a', 'l', 'm', 'o', 's']


In [5]:
char2idx = {char: idx for idx, char in enumerate(chars)}
idx2char = {idx: char for idx, char in enumerate(chars)}

In [8]:
x_data = [char2idx[char] for char in sequence[:-1]]
y_data = [char2idx[char] for char in sequence[1:]]

x = torch.tensor(x_data).unsqueeze(1)
y = torch.tensor(y_data)

x.shape, y.shape

(torch.Size([4, 1]), torch.Size([4]))

In [9]:
class HarfRNN(nn.Module):
  def __init__(self, vocab_size, hidden_size):
    super(HarfRNN, self).__init__()
    self.rnn = nn.RNN(vocab_size, hidden_size)
    self.fc = nn.Linear(hidden_size, vocab_size)

  def forward(self, x, hidden):
    out, hidden = self.rnn(x, hidden)
    out = self.fc(out.squeeze(1))
    return out, hidden



In [10]:
vocab_size = len(chars)
hidden_size = 8
model = HarfRNN(vocab_size, hidden_size)

In [11]:
def one_hot(index, vocab_size):
  vec = torch.zeros(1, 1, vocab_size)
  vec[0][0][index] = 1
  return vec

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [13]:
for epoch in range(100):
   loss = 0
   h = torch.zeros(1,1,hidden_size)

   for i in range(len(x)):
    input_vec = one_hot(x[i], vocab_size)
    output, h = model(input_vec, h.detach( ))
    loss += criterion(output, y[i].unsqueeze(0))

   optimizer.zero_grad()
   loss.backward()
   optimizer.step()

   if epoch % 10 == 0:
    pred_seq = ''
    h_test = torch.zeros(1,1,hidden_size)

    for i in x_data:
      input_vec = one_hot(i, vocab_size)
      output, h_test = model(input_vec, h_test)
      pred_idx = output.argmax().item()
      pred_char = idx2char[pred_idx]
      pred_seq += pred_char

    print(f'Epoch: {epoch}, Loss: {loss.item()}, Pred: {pred_seq}')

Epoch: 0, Loss: 6.584862232208252, Pred: mmmm
Epoch: 10, Loss: 4.869781494140625, Pred: alom
Epoch: 20, Loss: 3.0723588466644287, Pred: alom
Epoch: 30, Loss: 1.3091171979904175, Pred: alom
Epoch: 40, Loss: 0.5413163900375366, Pred: alom
Epoch: 50, Loss: 0.2641647160053253, Pred: alom
Epoch: 60, Loss: 0.15879419445991516, Pred: alom
Epoch: 70, Loss: 0.11214255541563034, Pred: alom
Epoch: 80, Loss: 0.08738254010677338, Pred: alom
Epoch: 90, Loss: 0.07206176221370697, Pred: alom
